In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_sku = df['Sku'].tolist()

print('总数量：' + str(len(list_sku)))
list_sku

总数量：2477


['Part # 121.04006',
 'Part # 121.07000',
 'Part # 121.07001',
 'Part # 121.07002',
 'Part # 121.07003',
 'Part # 121.07006',
 'Part # 121.07011',
 'Part # 121.07014',
 'Part # 121.07018',
 'Part # 121.10000',
 'Part # 121.10004',
 'Part # 121.10007',
 'Part # 121.10008',
 'Part # 121.10009',
 'Part # 121.11000',
 'Part # 121.11002',
 'Part # 121.11003',
 'Part # 121.11004',
 'Part # 121.11005',
 'Part # 121.11006',
 'Part # 121.11008',
 'Part # 121.11010',
 'Part # 121.20000',
 'Part # 121.20002',
 'Part # 121.20003',
 'Part # 121.20006',
 'Part # 121.20007',
 'Part # 121.20008',
 'Part # 121.20009',
 'Part # 121.20010',
 'Part # 121.20011',
 'Part # 121.20013',
 'Part # 121.20014',
 'Part # 121.20015',
 'Part # 121.20016',
 'Part # 121.20017',
 'Part # 121.20018',
 'Part # 121.20019',
 'Part # 121.20020',
 'Part # 121.20021',
 'Part # 121.20022',
 'Part # 121.20023',
 'Part # 121.20024',
 'Part # 121.20029',
 'Part # 121.20030',
 'Part # 121.20031',
 'Part # 121.20032',
 'Part # 121.

In [3]:
df = pd.DataFrame()

df

""


In [4]:
import requests

In [5]:
i = 0

sku = list_sku[i]

resp = requests.get('http://www.kakapart.com/oem/partscrossinterchange/centric-' + sku.replace('Part # ', '').replace('.', '') + '.html').text

resp

'<!DOCTYPE html>\r\n<html>\r\n    <head>\r\n        <title>未将对象引用设置到对象的实例。</title>\r\n        <meta name="viewport" content="width=device-width" />\r\n        <style>\r\n         body {font-family:"Verdana";font-weight:normal;font-size: .7em;color:black;} \r\n         p {font-family:"Verdana";font-weight:normal;color:black;margin-top: -5px}\r\n         b {font-family:"Verdana";font-weight:bold;color:black;margin-top: -5px}\r\n         H1 { font-family:"Verdana";font-weight:normal;font-size:18pt;color:red }\r\n         H2 { font-family:"Verdana";font-weight:normal;font-size:14pt;color:maroon }\r\n         pre {font-family:"Consolas","Lucida Console",Monospace;font-size:11pt;margin:0;padding:0.5em;line-height:14pt}\r\n         .marker {font-weight: bold; color: black;text-decoration: none;}\r\n         .version {color: gray;}\r\n         .error {margin-bottom: 10px;}\r\n         .expandable { text-decoration:underline; font-weight:bold; color:navy; cursor:pointer; }\r\n         @media sc

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(resp, 'lxml')

with open('./OE.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   未将对象引用设置到对象的实例。
  </title>
  <meta content="width=device-width" name="viewport"/>
  <style>
   body {font-family:"Verdana";font-weight:normal;font-size: .7em;color:black;} 
         p {font-family:"Verdana";font-weight:normal;color:black;margin-top: -5px}
         b {font-family:"Verdana";font-weight:bold;color:black;margin-top: -5px}
         H1 { font-family:"Verdana";font-weight:normal;font-size:18pt;color:red }
         H2 { font-family:"Verdana";font-weight:normal;font-size:14pt;color:maroon }
         pre {font-family:"Consolas","Lucida Console",Monospace;font-size:11pt;margin:0;padding:0.5em;line-height:14pt}
         .marker {font-weight: bold; color: black;text-decoration: none;}
         .version {color: gray;}
         .error {margin-bottom: 10px;}
         .expandable { text-decoration:underline; font-weight:bold; color:navy; cursor:pointer; }
         @media screen and (max-width: 639px) {
          pre { width: 440px; overflow: 

In [8]:
from lxml import etree

In [9]:
html = etree.HTML(str(soup))

list_tr = html.xpath('//table[@class="os-main-table"]/tbody/tr[@class="os-row"]')

list_tr

[]

In [10]:
dict_oe = {}
for tr in list_tr:
    factory = tr.xpath('./td/text()')[0].strip()
    cross = tr.xpath('./td/text()')[2].strip()
    if cross == 'Direct Cross Interchange':
        key = factory + '_Direct'
    elif cross == 'Indirect Cross Interchange':
        key = factory + '_Indirect'
    else:
        key = factory + '_NULL'
        
    if key not in dict_oe:
        dict_oe[key] = [tr.xpath('./td/text()')[1].strip()]
    else:
        dict_oe[key].append(tr.xpath('./td/text()')[1].strip())
        
dict_oe

{}

In [11]:
df_temp = pd.DataFrame([{'status': 'ok',
                         'Sku': sku}])

for key in dict_oe:
    df_temp.loc[0, 'OE_'+key] = ';'.join(dict_oe[key])

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

list_column = list(df.columns)[1:]
list_column.sort()
df = df[['Sku'] + list_column]

df

,status,Sku
0,ok,Part # 121.04006
